In [ ]:
// Create solvers and get resultsval solvers = arrayOfNulls<NetworkSolver>(2)solvers[0] = JMT(model).apply { setOptions("seed", 23000) }solvers[1] = CTMC(model).apply { setOptions("keep", false) }for (i in solvers.indices) {    val solver = solvers[i]    solver?.let {                try {            val avgTable = it.avgTable            avgTable.print()        } catch (e: Exception) {            println("Error computing results: ${e.message}")        }    }}

In [ ]:
// Set routing for both classes
nodes[0]?.setRouting(jobclasses[0], RoutingStrategy.RROBIN)
nodes[1]?.setProbRouting(jobclasses[0], nodes[0], 1.0)
nodes[2]?.setProbRouting(jobclasses[0], nodes[0], 1.0)

nodes[0]?.setRouting(jobclasses[1], RoutingStrategy.RROBIN)
nodes[1]?.setProbRouting(jobclasses[1], nodes[0], 1.0)
nodes[2]?.setProbRouting(jobclasses[1], nodes[0], 1.0)

In [ ]:
// Set service times
nodes[0]?.setService(jobclasses[0], HyperExp.fitMeanAndSCV(1.0, 25.0))
nodes[1]?.setService(jobclasses[0], map21)
nodes[2]?.setService(jobclasses[0], map31)

nodes[0]?.setService(jobclasses[1], HyperExp.fitMeanAndSCV(1.0, 25.0))
nodes[1]?.setService(jobclasses[1], map22)
nodes[2]?.setService(jobclasses[1], map32)

// Add links
model.addLink(nodes[0], nodes[0])
model.addLink(nodes[0], nodes[1])
model.addLink(nodes[0], nodes[2])
model.addLink(nodes[1], nodes[0])
model.addLink(nodes[2], nodes[0])

In [ ]:
// Define service processes
// Renewal processes
val map21 = APH(
    Matrix.of(doubleArrayOf(1.0, 0.0)),
    Matrix.of(arrayOf(
        doubleArrayOf(-2.0, 2.0),
        doubleArrayOf(0.0, -0.5)
    ))
)
val map22 = MAP(Matrix.of(arrayOf(doubleArrayOf(-1.0))), Matrix.of(arrayOf(doubleArrayOf(1.0))))

// Non-renewal processes  
val map31 = MAP(
    Matrix.of(arrayOf(
        doubleArrayOf(-20.0, 0.0),
        doubleArrayOf(0.0, -1.0)
    )),
    Matrix.of(arrayOf(
        doubleArrayOf(0.0, 20.0),
        doubleArrayOf(0.8, 0.2)
    ))
)
val map32 = MAP(
    Matrix.of(arrayOf(
        doubleArrayOf(-20.0, 0.0),
        doubleArrayOf(0.0, -1.0)
    )),
    Matrix.of(arrayOf(
        doubleArrayOf(0.0, 20.0),
        doubleArrayOf(0.8, 0.2)
    ))
)

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create model
val model = Network("model")

// Create nodes
val nodes = arrayOfNulls<Node>(3)
nodes[0] = Delay(model, "Delay")
nodes[1] = Queue(model, "Queue1", SchedStrategy.PS)
nodes[2] = Queue(model, "Queue2", SchedStrategy.FCFS)

// Create job classes
val jobclasses = arrayOfNulls<JobClass>(2)
jobclasses[0] = ClosedClass(model, "Class1", 1, nodes[0], 0)
jobclasses[1] = ClosedClass(model, "Class2", 2, nodes[0], 0)

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.util.matrix.Matrix

# State-Dependent Routing with Two Classes in Closed Network

This example demonstrates state-dependent routing in a closed queueing network with two job classes using MAP and APH service processes.